In [1]:
# Import Dependencies
import numpy as np

# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
import pandas as pd

In [2]:
# Import Csv

file_to_load = 'Resources/Substance_Abuse_Programs.csv'

# Convert to DF
programs_df = pd.read_csv(file_to_load)
programs_df

,X,Y,OBJECTID,source,ext_id,cat1,cat2,cat3,org_name,Name,...,zip,link,use_type,latitude,longitude,date_updated,email,dis_status,POINT_X,POINT_Y
0,-118.257443,34.128058,32,211.0,NaN,Health and Mental Health,Substance Abuse Programs,NaN,NaN,Glendale Memorial Hospital And Health Center,...,91204,http://egis3.lacounty.gov/lms/?p=570,publish,34.128058,-118.257443,2016/01/22 16:10:02+00,NaN,NaN,6.483758e+06,1.869083e+06
1,-118.265746,34.037364,62,211.0,NaN,Health and Mental Health,Substance Abuse Programs,NaN,Catholic Healthcare West,California Hospital Medical Center,...,90015,http://egis3.lacounty.gov/lms/?p=574,publish,34.037364,-118.265746,2016/01/21 17:29:05+00,NaN,NaN,6.481158e+06,1.836084e+06
2,-118.291013,34.098019,78,211.0,NaN,Health and Mental Health,Substance Abuse Programs,NaN,LA County,Children's Hospital Los Angeles,...,90027,http://egis3.lacounty.gov/lms/?p=576,publish,34.098019,-118.291013,2016/01/22 10:03:15+00,NaN,NaN,6.473567e+06,1.858179e+06
3,-118.186744,33.808107,88,211.0,NaN,Health and Mental Health,Substance Abuse Programs,NaN,Memorial Care Health Systems,Long Beach Memorial Medical Center,...,90806,http://egis3.lacounty.gov/lms/?p=577,publish,33.808107,-118.186744,2016/01/25 13:37:43+00,NaN,NaN,6.504938e+06,1.752599e+06
4,-118.307462,33.738067,110,211.0,NaN,Health and Mental Health,Substance Abuse Programs,NaN,california.providence.org/san-pedro/Pages/defa...,Providence Little Company Of Mary Medical Cent...,...,90732,http://egis3.lacounty.gov/lms/?p=591,publish,33.738067,-118.307462,2017/10/31 16:49:05+00,NaN,NaN,6.468187e+06,1.727199e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
452,-118.292002,33.967905,65696,211.0,NaN,Health and Mental Health,Substance Abuse Programs,NaN,Vermont Village Community Development Corporat...,Vermont Village Community Development Corporat...,...,90044,http://egis3.lacounty.gov/lms/?p=71882,publish,33.967905,-118.292002,2013/08/13 09:14:13+00,NaN,NaN,6.473130e+06,1.810829e+06
453,-118.277917,33.972323,65791,NaN,NaN,Health and Mental Health,Substance Abuse Programs,NaN,South Central Health and Rehabilitation Programs,South Central Health and Rehabilitation Programs,...,90003,http://egis3.lacounty.gov/lms/?p=72266,publish,33.972323,-118.277917,2013/12/18 08:56:30+00,NaN,NaN,6.477406e+06,1.812425e+06
454,-118.207198,33.904867,65812,NaN,NaN,Health and Mental Health,Substance Abuse Programs,NaN,Shields For Families Inc. - Tamar Village,Shields For Families Inc. - Tamar Village,...,90221,http://egis3.lacounty.gov/lms/?p=72322,publish,33.904867,-118.207198,2013/08/14 08:21:34+00,NaN,NaN,6.498797e+06,1.787824e+06
455,-118.339152,34.006534,65820,NaN,NaN,Health and Mental Health,Substance Abuse Programs,NaN,SA 6 Advisory Committee (SAAC) West Central Me...,SA 6 Advisory Committee (SAAC) West Central Me...,...,90008,http://egis3.lacounty.gov/lms/?p=72809,publish,34.006534,-118.339152,2013/12/18 09:37:24+00,NaN,NaN,6.458881e+06,1.824932e+06


In [3]:
# Examining the columns
programs_df.columns

Index(['X', 'Y', 'OBJECTID', 'source', 'ext_id', 'cat1', 'cat2', 'cat3',
       'org_name', 'Name', 'addrln1', 'addrln2', 'city', 'state', 'hours',
       'phones', 'url', 'info1', 'info2', 'post_id', 'description', 'zip',
       'link', 'use_type', 'latitude', 'longitude', 'date_updated', 'email',
       'dis_status', 'POINT_X', 'POINT_Y'],
      dtype='object')

In [4]:
# Drop Null columns
programs_df = programs_df.drop(['email', 'source', 'link', 'use_type', 'cat3', 'description', 'ext_id', 'addrln1', 'addrln2', 'info1', 'info2', 'hours', 'phones'],axis = 1)
programs_df.head(10)

,X,Y,OBJECTID,cat1,cat2,org_name,Name,city,state,url,post_id,zip,latitude,longitude,date_updated,dis_status,POINT_X,POINT_Y
0,-118.257443,34.128058,32,Health and Mental Health,Substance Abuse Programs,NaN,Glendale Memorial Hospital And Health Center,Glendale,CA,http://www.glendalememorial.com/,570,91204,34.128058,-118.257443,2016/01/22 16:10:02+00,NaN,6.483758e+06,1.869083e+06
1,-118.265746,34.037364,62,Health and Mental Health,Substance Abuse Programs,Catholic Healthcare West,California Hospital Medical Center,Los Angeles,CA,http://www.chmcla.org,574,90015,34.037364,-118.265746,2016/01/21 17:29:05+00,NaN,6.481158e+06,1.836084e+06
2,-118.291013,34.098019,78,Health and Mental Health,Substance Abuse Programs,LA County,Children's Hospital Los Angeles,Los Angeles,CA,http://www.chla.org/,576,90027,34.098019,-118.291013,2016/01/22 10:03:15+00,NaN,6.473567e+06,1.858179e+06
3,-118.186744,33.808107,88,Health and Mental Health,Substance Abuse Programs,Memorial Care Health Systems,Long Beach Memorial Medical Center,Long Beach,CA,http://memorialcare.org/locations/search/adult...,577,90806,33.808107,-118.186744,2016/01/25 13:37:43+00,NaN,6.504938e+06,1.752599e+06
4,-118.307462,33.738067,110,Health and Mental Health,Substance Abuse Programs,california.providence.org/san-pedro/Pages/defa...,Providence Little Company Of Mary Medical Cent...,San Pedro,CA,california.providence.org/san-pedro/Pages/defa...,591,90732,33.738067,-118.307462,2017/10/31 16:49:05+00,NaN,6.468187e+06,1.727199e+06
5,-118.537467,34.199490,121,Health and Mental Health,Substance Abuse Programs,www.tarzanatc.org,Tarzana Treatment Center - Reseda,Reseda,CA,www.tarzanatc.org,598,91335,34.199490,-118.537467,2013/06/01 11:50:56+00,NaN,6.399152e+06,1.895413e+06
6,-118.541090,34.180015,142,Health and Mental Health,Substance Abuse Programs,NaN,Tarzana Treatment Center - Youth Services,Tarzana,CA,www.tarzanatc.org,613,91356,34.180015,-118.541090,2013/06/01 11:50:56+00,NaN,6.398019e+06,1.888332e+06
7,-118.310283,33.928430,145,Health and Mental Health,Substance Abuse Programs,Shields For Families Inc. - Tamar Village,Shields For Families Inc. - Tamar Village,Los Angeles,CA,www.shieldsforfamilies.org,619,90047,33.928430,-118.310283,2016/05/16 08:38:19+00,NaN,6.467543e+06,1.796480e+06
8,-118.198412,33.795663,169,Health and Mental Health,Substance Abuse Programs,NaN,Tarzana Treatment Center - Long Beach Residential,Long Beach,CA,www.tarzanatc.org,634,90806,33.795663,-118.198412,2013/06/01 11:50:56+00,NaN,6.501385e+06,1.748077e+06
9,-118.147459,34.018557,189,Health and Mental Health,Substance Abuse Programs,NaN,Mela Counseling Services Center,Los Angeles,CA,www.melacounseling.org,644,90022,34.018557,-118.147459,2013/06/01 11:50:56+00,NaN,6.516985e+06,1.829166e+06


In [5]:
programs_df.count()

X               457
Y               457
OBJECTID        457
cat1            457
cat2            457
org_name        254
Name            457
city            457
state           457
url             335
post_id         457
zip             457
latitude        457
longitude       457
date_updated    457
dis_status        1
POINT_X         457
POINT_Y         457
dtype: int64

In [6]:
# Dropping additional columns
programs_df = programs_df.drop(['url', 'dis_status'], axis=1)
programs_df.head(10)

,X,Y,OBJECTID,cat1,cat2,org_name,Name,city,state,post_id,zip,latitude,longitude,date_updated,POINT_X,POINT_Y
0,-118.257443,34.128058,32,Health and Mental Health,Substance Abuse Programs,NaN,Glendale Memorial Hospital And Health Center,Glendale,CA,570,91204,34.128058,-118.257443,2016/01/22 16:10:02+00,6.483758e+06,1.869083e+06
1,-118.265746,34.037364,62,Health and Mental Health,Substance Abuse Programs,Catholic Healthcare West,California Hospital Medical Center,Los Angeles,CA,574,90015,34.037364,-118.265746,2016/01/21 17:29:05+00,6.481158e+06,1.836084e+06
2,-118.291013,34.098019,78,Health and Mental Health,Substance Abuse Programs,LA County,Children's Hospital Los Angeles,Los Angeles,CA,576,90027,34.098019,-118.291013,2016/01/22 10:03:15+00,6.473567e+06,1.858179e+06
3,-118.186744,33.808107,88,Health and Mental Health,Substance Abuse Programs,Memorial Care Health Systems,Long Beach Memorial Medical Center,Long Beach,CA,577,90806,33.808107,-118.186744,2016/01/25 13:37:43+00,6.504938e+06,1.752599e+06
4,-118.307462,33.738067,110,Health and Mental Health,Substance Abuse Programs,california.providence.org/san-pedro/Pages/defa...,Providence Little Company Of Mary Medical Cent...,San Pedro,CA,591,90732,33.738067,-118.307462,2017/10/31 16:49:05+00,6.468187e+06,1.727199e+06
5,-118.537467,34.199490,121,Health and Mental Health,Substance Abuse Programs,www.tarzanatc.org,Tarzana Treatment Center - Reseda,Reseda,CA,598,91335,34.199490,-118.537467,2013/06/01 11:50:56+00,6.399152e+06,1.895413e+06
6,-118.541090,34.180015,142,Health and Mental Health,Substance Abuse Programs,NaN,Tarzana Treatment Center - Youth Services,Tarzana,CA,613,91356,34.180015,-118.541090,2013/06/01 11:50:56+00,6.398019e+06,1.888332e+06
7,-118.310283,33.928430,145,Health and Mental Health,Substance Abuse Programs,Shields For Families Inc. - Tamar Village,Shields For Families Inc. - Tamar Village,Los Angeles,CA,619,90047,33.928430,-118.310283,2016/05/16 08:38:19+00,6.467543e+06,1.796480e+06
8,-118.198412,33.795663,169,Health and Mental Health,Substance Abuse Programs,NaN,Tarzana Treatment Center - Long Beach Residential,Long Beach,CA,634,90806,33.795663,-118.198412,2013/06/01 11:50:56+00,6.501385e+06,1.748077e+06
9,-118.147459,34.018557,189,Health and Mental Health,Substance Abuse Programs,NaN,Mela Counseling Services Center,Los Angeles,CA,644,90022,34.018557,-118.147459,2013/06/01 11:50:56+00,6.516985e+06,1.829166e+06


In [7]:
# Find and drop remaining null values

programs_df.isnull().sum()

X                 0
Y                 0
OBJECTID          0
cat1              0
cat2              0
org_name        203
Name              0
city              0
state             0
post_id           0
zip               0
latitude          0
longitude         0
date_updated      0
POINT_X           0
POINT_Y           0
dtype: int64

In [8]:
# Drop rows with null values

programs_df = programs_df.dropna(subset=['state', 'zip', 'org_name'])
programs_df.isnull().sum()

X               0
Y               0
OBJECTID        0
cat1            0
cat2            0
org_name        0
Name            0
city            0
state           0
post_id         0
zip             0
latitude        0
longitude       0
date_updated    0
POINT_X         0
POINT_Y         0
dtype: int64

In [9]:
programs_df.count()
programs_df.head(10)

,X,Y,OBJECTID,cat1,cat2,org_name,Name,city,state,post_id,zip,latitude,longitude,date_updated,POINT_X,POINT_Y
1,-118.265746,34.037364,62,Health and Mental Health,Substance Abuse Programs,Catholic Healthcare West,California Hospital Medical Center,Los Angeles,CA,574,90015,34.037364,-118.265746,2016/01/21 17:29:05+00,6.481158e+06,1.836084e+06
2,-118.291013,34.098019,78,Health and Mental Health,Substance Abuse Programs,LA County,Children's Hospital Los Angeles,Los Angeles,CA,576,90027,34.098019,-118.291013,2016/01/22 10:03:15+00,6.473567e+06,1.858179e+06
3,-118.186744,33.808107,88,Health and Mental Health,Substance Abuse Programs,Memorial Care Health Systems,Long Beach Memorial Medical Center,Long Beach,CA,577,90806,33.808107,-118.186744,2016/01/25 13:37:43+00,6.504938e+06,1.752599e+06
4,-118.307462,33.738067,110,Health and Mental Health,Substance Abuse Programs,california.providence.org/san-pedro/Pages/defa...,Providence Little Company Of Mary Medical Cent...,San Pedro,CA,591,90732,33.738067,-118.307462,2017/10/31 16:49:05+00,6.468187e+06,1.727199e+06
5,-118.537467,34.199490,121,Health and Mental Health,Substance Abuse Programs,www.tarzanatc.org,Tarzana Treatment Center - Reseda,Reseda,CA,598,91335,34.199490,-118.537467,2013/06/01 11:50:56+00,6.399152e+06,1.895413e+06
7,-118.310283,33.928430,145,Health and Mental Health,Substance Abuse Programs,Shields For Families Inc. - Tamar Village,Shields For Families Inc. - Tamar Village,Los Angeles,CA,619,90047,33.928430,-118.310283,2016/05/16 08:38:19+00,6.467543e+06,1.796480e+06
11,-118.259793,34.077292,200,Health and Mental Health,Substance Abuse Programs,www.elcentrodelpueblo.org,El Centro Del Pueblo,Los Angeles,CA,649,90026,34.077292,-118.259793,2013/06/01 11:50:56+00,6.483000e+06,1.850610e+06
12,-118.540269,34.179848,219,Health and Mental Health,Substance Abuse Programs,www.tarzanatc.org,Tarzana Treatment Center Inc,Tarzana,CA,660,91356,34.179848,-118.540269,2013/06/01 11:50:56+00,6.398267e+06,1.888270e+06
15,-118.225448,33.923299,313,Health and Mental Health,Substance Abuse Programs,www.shieldsforfamilies.org,Shields For Families Inc - Eden,Lynwood,CA,712,90262,33.923299,-118.225448,2013/06/01 11:50:56+00,6.493274e+06,1.794544e+06
21,-118.151838,34.031363,443,Health and Mental Health,Substance Abuse Programs,www.bienvenidos.org,Bienvenidos - East Los Angeles Family Preserva...,Los Angeles,CA,799,90022,34.031363,-118.151838,2013/06/01 11:50:56+00,6.515665e+06,1.833829e+06


In [10]:
programs_df.count()
# Only include mental health centers listed in Los Angeles
#programs_df = centers_df.loc[centers_df['city']== 'Los Angeles']
#programs_df.head(10)


X               254
Y               254
OBJECTID        254
cat1            254
cat2            254
org_name        254
Name            254
city            254
state           254
post_id         254
zip             254
latitude        254
longitude       254
date_updated    254
POINT_X         254
POINT_Y         254
dtype: int64

In [11]:
programs_df.count()

X               254
Y               254
OBJECTID        254
cat1            254
cat2            254
org_name        254
Name            254
city            254
state           254
post_id         254
zip             254
latitude        254
longitude       254
date_updated    254
POINT_X         254
POINT_Y         254
dtype: int64

In [12]:
programs_df.head(10)

,X,Y,OBJECTID,cat1,cat2,org_name,Name,city,state,post_id,zip,latitude,longitude,date_updated,POINT_X,POINT_Y
1,-118.265746,34.037364,62,Health and Mental Health,Substance Abuse Programs,Catholic Healthcare West,California Hospital Medical Center,Los Angeles,CA,574,90015,34.037364,-118.265746,2016/01/21 17:29:05+00,6.481158e+06,1.836084e+06
2,-118.291013,34.098019,78,Health and Mental Health,Substance Abuse Programs,LA County,Children's Hospital Los Angeles,Los Angeles,CA,576,90027,34.098019,-118.291013,2016/01/22 10:03:15+00,6.473567e+06,1.858179e+06
3,-118.186744,33.808107,88,Health and Mental Health,Substance Abuse Programs,Memorial Care Health Systems,Long Beach Memorial Medical Center,Long Beach,CA,577,90806,33.808107,-118.186744,2016/01/25 13:37:43+00,6.504938e+06,1.752599e+06
4,-118.307462,33.738067,110,Health and Mental Health,Substance Abuse Programs,california.providence.org/san-pedro/Pages/defa...,Providence Little Company Of Mary Medical Cent...,San Pedro,CA,591,90732,33.738067,-118.307462,2017/10/31 16:49:05+00,6.468187e+06,1.727199e+06
5,-118.537467,34.199490,121,Health and Mental Health,Substance Abuse Programs,www.tarzanatc.org,Tarzana Treatment Center - Reseda,Reseda,CA,598,91335,34.199490,-118.537467,2013/06/01 11:50:56+00,6.399152e+06,1.895413e+06
7,-118.310283,33.928430,145,Health and Mental Health,Substance Abuse Programs,Shields For Families Inc. - Tamar Village,Shields For Families Inc. - Tamar Village,Los Angeles,CA,619,90047,33.928430,-118.310283,2016/05/16 08:38:19+00,6.467543e+06,1.796480e+06
11,-118.259793,34.077292,200,Health and Mental Health,Substance Abuse Programs,www.elcentrodelpueblo.org,El Centro Del Pueblo,Los Angeles,CA,649,90026,34.077292,-118.259793,2013/06/01 11:50:56+00,6.483000e+06,1.850610e+06
12,-118.540269,34.179848,219,Health and Mental Health,Substance Abuse Programs,www.tarzanatc.org,Tarzana Treatment Center Inc,Tarzana,CA,660,91356,34.179848,-118.540269,2013/06/01 11:50:56+00,6.398267e+06,1.888270e+06
15,-118.225448,33.923299,313,Health and Mental Health,Substance Abuse Programs,www.shieldsforfamilies.org,Shields For Families Inc - Eden,Lynwood,CA,712,90262,33.923299,-118.225448,2013/06/01 11:50:56+00,6.493274e+06,1.794544e+06
21,-118.151838,34.031363,443,Health and Mental Health,Substance Abuse Programs,www.bienvenidos.org,Bienvenidos - East Los Angeles Family Preserva...,Los Angeles,CA,799,90022,34.031363,-118.151838,2013/06/01 11:50:56+00,6.515665e+06,1.833829e+06


In [13]:
print(programs_df['city'].unique())

['Los Angeles' 'Long Beach' 'San Pedro' 'Reseda' 'Tarzana' 'Lynwood'
 'Pico Rivera' 'Pasadena' 'Venice' 'Lawndale' 'Newhall' 'San Gabriel'
 'Pomona' 'Inglewood' 'West Covina' 'Wilmington' 'Canoga Park' 'La Mirada'
 'El Monte' 'Compton' 'Lake View Terrace' 'Covina' 'Sunland' 'Lancaster'
 'Torrance' 'Acton' 'Burbank' 'Panorama City' 'Santa Clarita' 'Van Nuys'
 'Hollywood' 'Whittier' 'Downey' 'Bell Gardens' 'Bellflower' 'Cerritos'
 'Gardena' 'Arcadia' 'San Fernando' 'Saugus' 'Hawthorne' 'North Hollywood'
 'Chatsworth' 'Century City' 'Woodland Hills' 'Beverly Hills' 'Alhambra'
 'Sylmar' 'North Hills' 'Carson' 'Santa Monica' 'Culver City' 'Glendale'
 'Hermosa Beach' 'Glendora' 'La Puente' 'Los Angeles ']
